In [14]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("random-forest-hyperopt-hw2")

<Experiment: artifact_location='/Users/mmukherjee/Documents/LearningAndDevelopment/DataTalksClub/mlops-zoomcamp/cohorts/2023/02-experiment-tracking/homework/hw2-artifacts/4', creation_time=1684858857467, experiment_id='4', last_update_time=1684858857467, lifecycle_stage='active', name='random-forest-hyperopt-hw2', tags={}>

In [89]:
experiment = mlflow.get_experiment_by_name("random-forest-hyperopt")

In [90]:
experiment.experiment_id

'5'

In [91]:
client = MlflowClient()

In [92]:
runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=5,
        order_by=["metrics.validation_RMSE ASC"]
    )

In [107]:
HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
top_n = 5
experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
runs = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=top_n,
    order_by=["metrics.validation_RMSE ASC"]
)

In [112]:
for sl_id, run in enumerate(runs):
    print(f"{sl_id}. rid: {run.info.run_id}, rname: {run.info.run_name}, rmse: {round(run.data.metrics['validation_RMSE'],3)}")
    print(f"params: {run.data.params}")
    print("-------------------")

0. rid: 9c8920bdd6e3423bb0e06a7703020531, rname: awesome-lark-606, rmse: 2.45
params: {'n_estimators': '34', 'max_depth': '15', 'min_samples_split': '2', 'min_samples_leaf': '4', 'random_state': '42', 'n_jobs': '-1'}
-------------------
1. rid: a23914763620454f88492d8300c3576c, rname: dashing-tern-16, rmse: 2.451
params: {'n_estimators': '25', 'max_depth': '20', 'min_samples_split': '8', 'min_samples_leaf': '3', 'random_state': '42', 'n_jobs': '-1'}
-------------------
2. rid: 5c44b93ef33b4dfba4232c864d467db7, rname: placid-seal-345, rmse: 2.452
params: {'n_estimators': '28', 'max_depth': '16', 'min_samples_split': '3', 'min_samples_leaf': '3', 'random_state': '42', 'n_jobs': '-1'}
-------------------
3. rid: 51b90335a91a4a42a1b156547b72c597, rname: kindly-whale-362, rmse: 2.454
params: {'n_estimators': '22', 'max_depth': '11', 'min_samples_split': '5', 'min_samples_leaf': '2', 'random_state': '42', 'n_jobs': '-1'}
-------------------
4. rid: ee7f0e9078c24efea68e8e7fb54eba1f, rname: le

In [128]:
import os
import pickle
import click
import mlflow

from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

In [135]:
def train_and_log_model(data_path, params):
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))
    X_test, y_test = load_pickle(os.path.join(data_path, "test.pkl"))

#     with mlflow.start_run():
    for param in RF_PARAMS:
        params[param] = int(params[param])

    rf = RandomForestRegressor(**params)
    rf.fit(X_train, y_train)

    # Evaluate model on the validation and test sets
    val_rmse = mean_squared_error(y_val, rf.predict(X_val), squared=False)
#     mlflow.log_metric("val_rmse", val_rmse)
    
    test_rmse = mean_squared_error(y_test, rf.predict(X_test), squared=False)
#     mlflow.log_metric("test_rmse", test_rmse)
    print(f"val_rmse: {round(val_rmse,3)}, test_rmse: {round(test_rmse,3)}")

In [121]:
data_path = "../data/output"
X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
X_train.shape

(59603, 6683)

In [122]:
r0 = runs[0]

In [124]:
r0.data.params

{'n_estimators': '34',
 'max_depth': '15',
 'min_samples_split': '2',
 'min_samples_leaf': '4',
 'random_state': '42',
 'n_jobs': '-1'}

In [136]:
for sl_id, run in enumerate(runs):
    print(f"{sl_id}. rid: {run.info.run_id}, rname: {run.info.run_name}, rmse: {round(run.data.metrics['validation_RMSE'],3)}")
    print(f"params: {run.data.params}")
    train_and_log_model(data_path, run.data.params)
    print("-------------------")
    

0. rid: 9c8920bdd6e3423bb0e06a7703020531, rname: awesome-lark-606, rmse: 2.45
params: {'n_estimators': 34, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 4, 'random_state': 42, 'n_jobs': -1}
val_rmse: 2.45, test_rmse: 2.285
-------------------
1. rid: a23914763620454f88492d8300c3576c, rname: dashing-tern-16, rmse: 2.451
params: {'n_estimators': 25, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3, 'random_state': 42, 'n_jobs': -1}
val_rmse: 2.451, test_rmse: 2.295
-------------------
2. rid: 5c44b93ef33b4dfba4232c864d467db7, rname: placid-seal-345, rmse: 2.452
params: {'n_estimators': 28, 'max_depth': 16, 'min_samples_split': 3, 'min_samples_leaf': 3, 'random_state': 42, 'n_jobs': -1}
val_rmse: 2.452, test_rmse: 2.292
-------------------
3. rid: 51b90335a91a4a42a1b156547b72c597, rname: kindly-whale-362, rmse: 2.454
params: {'n_estimators': 22, 'max_depth': 11, 'min_samples_split': 5, 'min_samples_leaf': 2, 'random_state': 42, 'n_jobs': -1}
val_rmse: 2.454, t

In [94]:
r0 = runs[0]

In [95]:
r0.info

<RunInfo: artifact_uri='/Users/mmukherjee/Documents/LearningAndDevelopment/DataTalksClub/mlops-zoomcamp/cohorts/2023/02-experiment-tracking/homework/hw2-artifacts/5/9c8920bdd6e3423bb0e06a7703020531/artifacts', end_time=1684923289112, experiment_id='5', lifecycle_stage='active', run_id='9c8920bdd6e3423bb0e06a7703020531', run_name='awesome-lark-606', run_uuid='9c8920bdd6e3423bb0e06a7703020531', start_time=1684923288374, status='FINISHED', user_id='mmukherjee'>

In [96]:
r0.data

<RunData: metrics={'validation_RMSE': 2.449827329704216}, params={'max_depth': '15',
 'min_samples_leaf': '4',
 'min_samples_split': '2',
 'n_estimators': '34',
 'n_jobs': '-1',
 'random_state': '42'}, tags={'mlflow.runName': 'awesome-lark-606',
 'mlflow.source.git.commit': '843cbc1ee15d3c1b40d9a95ff76e559d7bcfa67b',
 'mlflow.source.name': 'hpo.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'mmukherjee'}>

In [97]:
r0.data.params

{'n_estimators': '34',
 'max_depth': '15',
 'min_samples_split': '2',
 'min_samples_leaf': '4',
 'random_state': '42',
 'n_jobs': '-1'}

In [98]:
r0.data.metrics

{'validation_RMSE': 2.449827329704216}

In [71]:
client.restore_experiment(3)

In [68]:
# !mlflow experiments restore --experiment-id 3

Usage: mlflow experiments restore [OPTIONS]

  Restore a deleted experiment. This also applies to experiment's metadata,
  runs and associated data. The command throws an error if the experiment is
  already active, cannot be found, or permanently deleted.

Options:
  -x, --experiment-id TEXT  [required]
  --help                    Show this message and exit.


In [85]:
deleted_experiment = mlflow.get_experiment_by_name("random-forest-hyperopt")

In [86]:
deleted_experiment

<Experiment: artifact_location='/Users/mmukherjee/Documents/LearningAndDevelopment/DataTalksClub/mlops-zoomcamp/cohorts/2023/02-experiment-tracking/homework/hw2-artifacts/5', creation_time=1684923287228, experiment_id='5', last_update_time=1684923287228, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>

In [99]:
params = r0.data.params
params

{'n_estimators': '34',
 'max_depth': '15',
 'min_samples_split': '2',
 'min_samples_leaf': '4',
 'random_state': '42',
 'n_jobs': '-1'}

In [100]:
RF_PARAMS = ['max_depth', 'n_estimators', 'min_samples_split', 'min_samples_leaf', 'random_state', 'n_jobs']

In [103]:
for param in RF_PARAMS:
    params[param] = int(params[param])

In [104]:
params

{'n_estimators': 34,
 'max_depth': 15,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'random_state': 42,
 'n_jobs': -1}

In [88]:
mlflow.get_experiment_by_name("random-forest-hyperopt")


<Experiment: artifact_location='/Users/mmukherjee/Documents/LearningAndDevelopment/DataTalksClub/mlops-zoomcamp/cohorts/2023/02-experiment-tracking/homework/hw2-artifacts/5', creation_time=1684923287228, experiment_id='5', last_update_time=1684923287228, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>

In [105]:
len(runs)

5

"""
to register the model you can use the method mlflow.register_model and 
you will need to pass the right model_uri in the form of a string that looks like this: "runs:/<RUN_ID>/model", 
and the name of the model (make sure to choose a good one!).
"""

In [ ]:
mlflow.register_model(register_model=f"runs:/{run_id}/model", name="best-model-rf-hw2")